# Encoder-Decoder implementation based on DL4MT

In [ ]:
import numpy as np
model = np.load("model_hal.npz")

for matrix in model:
    print(matrix, model[matrix].shape)

# Encoder

In [17]:
print ('Wemb:', model['Wemb'].shape)
for matrix in model:
    if matrix.startswith("encoder"):
        print(matrix, model[matrix].shape)


Wemb: (30000, 512)
encoder_U (1024, 2048)
encoder_W (512, 2048)
encoder_r_Wx (512, 1024)
encoder_bx (1024,)
encoder_b (2048,)
encoder_r_bx (1024,)
encoder_r_U (1024, 2048)
encoder_r_b (2048,)
encoder_r_W (512, 2048)
encoder_Ux (1024, 1024)
encoder_Wx (512, 1024)
encoder_r_Ux (1024, 1024)


## Common

* $\overline{E}$ - `Wemb` - source word embeddings, common for both directions, size ${K_x \times m}$, where $K_x = 30000$ i $m = 512$
* $m$: embedding size (e.g. 512)
* $n$: internal state size (e.g. 1024)

## Forward pass

* $\overrightarrow{W}_x$ - `encoder_Wx` $m\times n$
* $\overrightarrow{U}_x$ - `encoder_Ux`, size $n \times n$
* $\overrightarrow{b}_x$ - `encoder_bx`, size $n$
* $\overrightarrow{W}$ - `encoder_W`, size $m \times 2n$
* $\overrightarrow{U}$ - `encoder_U`, size $n\times 2n$
* $\overrightarrow{b}$ - `encoder_b`, size $2n$

## Backward pass
Analogously, with `_r_` as an interfix.

## Computation

Differences in comparing with the model from  Bahdanau:
* different place for bias.
* The reset $r_i$ and update $u_i$ vectors are computed together (they are concatenated).

$$
\renewcommand{\ora}[1]{\overrightarrow{#1}}
\renewcommand{\ola}[1]{\overleftarrow{#1}}
\ora{h}_i = \left\{
\begin{array}{ll}
    \ora{u}_i \circ \ora{h}_{i-1} + (1- \ora{u}_i) \circ \ora{\underline{h}}_i & \mathrm{, if ~ } i > 0 \\
0 & \mathrm{, if  ~} i = 0 
\end{array}
\right.
$$

where 

$$
\begin{eqnarray}
\left[
\begin{array}{c}
\ora{r}_i \\
\ora{u}_i\end{array}
\right] &=& \sigma\left(\ora{h}_{i-1}\ora{U} + \overline{E}_i\ora{W} + \ora{b} \right)\\
\ora{\underline{h}}_i &=& \tanh\left((\ora{h}_{i-1}\ora{U_x})  \circ  \ora{r}_i+ \overline{E}_i\ora{W_x} + \ora{b_x} \right)\\
\end{eqnarray}
$$

The backward pass is similar. The pass over the words is reversed, but the implementation stays the same.

For every word,  the $\ora{h}_i$ and $\ola{h}_i$ are concatenated into $h_i$:

$$
h_i = \left[
\begin{array}{c}
\ora{h}_i \\
\ola{h}_i
\end{array}
\right]
$$

and the context matrix - $c$
$$
c = \left[ h_1, \ldots, h_n\right]
$$

# Decoder 

In [11]:
for matrix in model:
    if matrix.startswith("decoder"):
        print(matrix, model[matrix].shape)

decoder_U (1024, 2048)
decoder_W (512, 2048)
decoder_b (2048,)
decoder_Wc (2048, 2048)
decoder_b_att (2048,)
decoder_bx_nl (1024,)
decoder_Wcx (2048, 1024)
decoder_Ux (1024, 1024)
decoder_bx (1024,)
decoder_Wc_att (2048, 2048)
decoder_U_att (2048, 1)
decoder_c_tt (1,)
decoder_U_nl (1024, 2048)
decoder_W_comb_att (1024, 2048)
decoder_b_nl (2048,)
decoder_Wx (512, 1024)
decoder_Ux_nl (1024, 1024)


# Decoder RNN

* $E_t$ - `Wemb_dec` - target language embeddings, size: ${K_y \times m}$, where $K_y = 30000$ i $m = 512$
* $m$: embedding size (e.g. 512)
* $n$: internal state size (e.g. 1024)

## Initialising decoder RNN

In the Bahdanau Model, the last $h_i$ was taken to compute an initial state for the decoder.

This model takes the mean $h$ of all $h_i$.

$$
\qquad s_0 = \tanh\left(hW_I + b_I\right)
$$

* $W_I$ - `ff_state_W` - size ${2n \times n}$
* $b_I$ - `ff_state_b` - size ${n}$

## Computing a new RNN decoder state

Lets define $E_i$ as the embedding vector of a word $y_i$. In other words,  $E_i = Ey_i$


 * $E$ - `Wemb_dec` - target word embeddings, size: ${K_y \times m}$, where $K_y = 30000$ and $m = 512$

The computation of the next state is divided into two steps: computing a middle state, which goes to the attention model and computing the genue state based on the attention model output.

## Computing the hidden state (First GRU layer?)

$$
\begin{eqnarray}
\left[
\begin{array}{c}
\ora{r}_i^h \\
\ora{u}_i^h\end{array}
\right] &=& \sigma \left( s_{i-1}U + E_{i-1}W + b \right) \\
\\
\overline{s}_i &=& \tanh \left( (s_{i-1}U_x) \circ r_i^h +  E_{i-1}W_x + b_x \right) \\
\\
s_i &=& u_i^h \circ s_{i-1} + (1- u_i^h) \circ \overline{s}_i \\
\end{eqnarray}
$$


**Computing reset and update vectors:**
 * $ s_{i-1}$ - the previous decoder state, size: $n$
 * $ E_{i-1}$ the embedding of the word $y_{i-1}$,
 * $U$ - `decoder_U` - a matrix for a state, size: ${n \times 2n}$
 * $W$ - `decoder_W` - a matrix for a word embedding, size: ${m \times 2n}$
 * $b$ - `decoder_b` - a bias vector, size: ${2n}$

**Computing a new hidden state:**
 * $U_x$ - `decoder_Ux` - a matrix for the previous decoder state, size: ${n \times n}$
 * $W_x$ - `decoder_Wx` - a matrix for a word embedding, size: ${m \times n}$
 * $b_x$ - `decoder_bx` - a bias vector, size: ${n}$

## Attention model (or Alignment model)

$$
c_i = \sum_{j=1}^{T_x} \alpha_{ij}h_j
$$

(or if $h$ is a state matrix for an entire batch $c = Ah$ where $A = \left[a_{ij}\right]$)

where

$$ 
\begin{eqnarray}
\alpha_{ij} &=& \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x}\exp(e_{ik})}
\end{eqnarray}
$$

$$
\begin{eqnarray}
e_{ij} &=& v_\alpha^T \tanh\left(s_{i} W_{\alpha} + b_{\alpha} + h_jU_{\alpha}\right) + c_{\alpha}
\end{eqnarray}
$$

When doing batch computation, the sum in the last step involves rather complicated broadcasting to 3D tensors to get matching shapes.


* $W_{\alpha}$ - `decoder_W_comb_att` - size: ${n \times 2n}$,
* $b_{\alpha}$ - `decoder_b_att` - size: $2n$,
* $U_{\alpha}$ - `decoder_Wc_att` - size: ${2n \times 2n}$,
* $v_{\alpha}$ - `decoder_U_att` - size: ${2n}$,
* $c_{\alpha}$ - `decoder_c_tt` - a scalar (normalisation constant (?)),

## Computing the final decoder state (Second GRU layer?)

Take care for the different bias in the computation of the intermediate state, $\tilde{z}_i$. This is an oddity in the way Nematus implements GRUs.

$$
\begin{eqnarray}
\left[
\begin{array}{c}
\ora{r}_i^f \\
\ora{u}_i^f\end{array}
\right] &=& \sigma \left( s_iU + c_iW + b  \right) \\
\\
\tilde{z}_i &=& \tanh\left( (s_iU_x + b_x) \circ r_i^f + c_iW_x \right) \\
\\
z_i &=& u_i^f \circ s_i + (1 - u_i^f) \circ \tilde{z}_i
\end{eqnarray}
$$

**Computing the reset and update vectors:**
 * $U$ - `decoder_U_nl` - matrix for the state, size: ${n \times 2n}$
 * $W$ - `decoder_Wc` - matrix for the context vector , size: ${m \times 2n}$
 * $b$ - `decoder_b_nl` - bias vector, size: ${ 2n}$

**Computing the next state**
 * $U_x$ - `decoder_Ux_nl` - matrix for a middle state, size: ${n \times n}$
 * $W_x$ - `decoder_Wcx` - matrix for the context vector, size: ${m \times n}$
 * $b_x$ - `decoder_bx_nl` - bias vector, size: ${n}$

## ReadOut or Deep Output or just getting probabilities over words

$$
\begin{eqnarray}
 t_i &=&\tanh \left( \left( z_iW_1 + b_1 \right)  + \left( E_{i-1} W_2 + b_2 \right) + \left( c_iW_3 + b_3 \right) \right) \\
\\
p(y_i|z_{i},y_{i-1},c_i) &=& \textrm{softmax} \left(  t_iW_4 + b_4 \right) \\
\end{eqnarray}
$$

* $W_1$ - `ff_logit_lstm_W` - size: ${n} \times {m}$ 
* $b_1$ - `ff_logit_lstm_b` - size: ${m} $
* $W_2$ - `ff_logit_prev_W` - v ${m} \times {m}$ 
* $b_2$ - `ff_logit_prev_b` - size: ${m} $
* $W_3$ - `ff_logit_ctx_W` - size: ${2n} \times {m}$ 
* $b_3$ - `ff_logit_ctx_b` - size: ${m} $
* $W_4$ - `ff_logit_W` - size: ${m} \times K_y$ 
* $b_4$ - `ff_logit_b` - size: $K_y $